# Benchmark

In [ ]:
!python -m pip install pandas plotly pyyaml

In [ ]:
import pandas as pd, numpy as np
import os, glob, datetime, time
import plotly as plotly
import plotly.express as px
import hjson
import yaml

In [ ]:
# Compile hardware for Questa (vsim)
!questa-2022.3 make bin/snitch_cluster.vsim

In [ ]:
# Compile software
!make DEBUG=ON sw

In [ ]:
# Post process traces
!make -j traces
!make logs/perf.csv

In [ ]:
# Read profile data
perf = pd.read_csv('logs/perf.csv', index_col=0)
perf.filter(regex=("1_."))

In [ ]:
fig = px.scatter(perf, y=['1_total_ipc', '1_fpss_occupancy', '1_fpss_fpu_occupancy', '1_snitch_occupancy'])
fig.update_layout(yaxis_range=[0,1])

In [ ]:
def run(cmd, env=None, dryrun=False):
    if dryrun:
        print(cmd)
    else:
        p = subprocess.Popen(cmd, env=env, shell=True)
        retcode = p.wait()
        if retcode != 0:
            sys.exit(retcode)

In [ ]:
!f'make CFG_OVERRIDE={cfg_file} rtl'

# Benchmark Configuration

In [ ]:
# Load top-level benchmark config
bench_config_name = "bench/bench.yaml"
with open(bench_config_name) as f:
    bench_config = yaml.load(f, Loader=yaml.Loader)
bench_config

In [ ]:
# flatten into a table
hw = pd.json_normalize(bench_config['hw']).add_prefix('hw.')
sw = pd.json_normalize(bench_config['sw']).add_prefix('sw.')

configs = hw.merge(sw, how='cross')

# Evaluate expressions
eval_cols = configs.filter(regex=(".*\.eval")).columns.tolist()
eval_cols_short = [x.removesuffix('.eval') for x in eval_cols]
for i, col in enumerate(eval_cols):
    short = eval_cols_short[i]
    print(short)
    configs[col] = configs[col].apply(lambda x: eval(x) if type(x) == str else x)            
        
configs = configs.rename(dict(zip(eval_cols, eval_cols_short)), axis=1)

# Explode arrays
for col in configs.columns.tolist():
    if 'sweep' in col:
        configs = configs.explode(col)
configs.reset_index(inplace=True, drop=True)
configs

In [ ]:
def compile_hw(config: str):
    print(f'Compiling hw with config: {config}')

def compile_sw(config: str, overrides):
    print(f'Compiling sw with config: {config} with overrides: \n{overrides}')

In [ ]:
for config_hw_name, config_hw in configs.groupby(by='hw.config'):
    
    compile_hw(config_hw_name)
    for config_sw_name, config_sw in config_hw.groupby(by='sw.name'):
        
        override_cols = configs.filter(regex=(f'sw\.{config_sw_name}.*')).columns.tolist()
        print(f'app: {config_sw_name}, {override_cols}')
        
        for a, b in config_hw.groupby(by=override_cols) if override_cols else []:
            compile_sw(a, b)
            # run_test()
            # post_process()